In [3]:
import requests
import configparser
import pandas as pd
from tqdm import tqdm
import time

In [4]:
def get_drive_time(origin, destination, access_token):
    url = "https://api.mapbox.com/directions/v5/mapbox/driving/{0},{1};{2},{3}".format(origin[0], origin[1], destination[0], destination[1])
    params = {
        "access_token": access_token,
        "depart_at": '2024-10-21T17:00'
    }
    try:
        response = requests.get(url, params = params)
        data = response.json()
        if "routes" in data and len(data["routes"]) > 0:
            duration = data["routes"][0]["duration"] / 60
            return duration
        else:
            return None
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

In [5]:
# Test usage
origin = (-0.39847, 51.56422)
destination = (-0.39014, 51.55711)

parser = configparser.ConfigParser()
parser.read("config.txt")
access_token = parser.get("API_credentials", "secret_token")

drive_time = get_drive_time(origin, destination, access_token)
print("Drive time: ", drive_time, " minutes")

Drive time:  4.848733333333333  minutes


In [ ]:
# Read in data
df = pd.read_csv("Catchment Data.csv")
df['DriveTime'] = None
df.to_csv("Catchment Data DTs.csv", index = False)

# Load existing progress
temp_df = pd.read_csv("Catchment Data DTs.csv")
processed_rows = temp_df.dropna(subset=['DriveTime']).shape[0]

# Initialize drive_times list with existing progress
drive_times = temp_df['DriveTime'].tolist()

# Continue processing from the last processed row
for i, (index, row) in enumerate(tqdm(df.iloc[processed_rows:].iterrows(), total=df.shape[0] - processed_rows)):
    origin = (row['CustomerLong'], row['CustomerLat'])
    destination = (row['CentreLong'], row['CentreLat'])
    duration = get_drive_time(origin, destination, access_token)
    drive_times.append(duration)
    temp_df.at[index, 'CentreName'] = row['CentreName']
    temp_df.at[index, 'CentreLat'] = row['CentreLat']
    temp_df.at[index, 'CentreLong'] = row['CentreLong']
    temp_df.at[index, 'CustomerLat'] = row['CustomerLat']
    temp_df.at[index, 'CustomerLong'] = row['CustomerLong']
    temp_df.at[index, 'TotalTickets'] = row['TotalTickets']
    temp_df.at[index, 'TotalValue'] = row['TotalValue']
    temp_df.at[index, 'DriveTime'] = duration
    time.sleep(0.2)

    # Save the CSV every 500 iterations
    if (i + 1) % 500 == 0:
        temp_df.to_csv('Catchment Data DTs.csv', index=False)

# Save any remaining rows after the loop finishes
temp_df.to_csv('Catchment Data DTs.csv', index=False)


  4%|▍         | 2844/70291 [17:55<6:48:41,  2.75it/s] 